# GQA的手撕
话不多说，直接开始



In [ ]:
# 每日一导
import torch
import torch.nn as nn
import torch.nn.functional as F
import math 

In [ ]:
class GroupedQueryAttention(nn.Module):
    def __init__(self,d_model,n_heads,n_kv_heads):
        super().__init__()
        assert d_model % n_heads == 0
        assert n_heads % n_kv_heads == 0

        self.d_model = d_model
        self.n_heads = n_heads
        self.n_kv_heads = n_kv_heads
        self.head_dim = d_model //n_heads
        # 计算每个分组的大小
        self.group_size = n_heads // n_kv_heads 

        # 初始化4个Linear
        self.w_q = nn.Linear(d_model,n_heads * self.head_dim)
        self.w_k = nn.Linear(d_model)
        self.w_v
        
